# Install Java (OSK)

at least Java 7.  
recommended Oracle JDK version 1.8.0_73  
JRE download: http://www.oracle.com/technetwork/java/javase/downloads/jre8-downloads-2133155.html  (2016.07.18)

# Install homebrew (OSX)

# Install Elasticsearch server (Ubuntu, CentOS, Windows)
https://www.elastic.co/downloads/elasticsearch

# Install Elasticsearch server (OSX)
https://gist.github.com/jpalala/ab3c33dd9ee5a6efbdae

# Install Elasticsearch client Python packages

# Start/Stop Elasticsearch server
https://robots.thoughtbot.com/starting-and-stopping-background-services-with-homebrew

# Check Elasticsearch server

# Install Python packages

# install Elasticsearch HQ (web admin plugin)

# ------------------- 수업 시작 ----------------------

# Elasticsearch vs RDBMS
http://www.slideshare.net/sophistlv/2-d1-elasticsearch
- Index = Database
- Type = Table
- Mapping = Schema
- Document = Row
- _id = Primary Key
- shard = pyisical partition

# Elasticsearch Reference (v2.3)
https://www.elastic.co/guide/en/elasticsearch/reference/current/index.html

# Change node name

# Docs Samples

In [1]:
import pickle
import pprint

docs = pickle.load(open('famous_saying.pkl', 'rb'))
print(len(docs))
pprint.pprint(docs[:10])

1000
[{'content': '오직 남들을 위하여 산 인생만이 가치 있는 것이다.', 'speaker': '아인슈타인'},
 {'content': '울지 않는 청년은 야만인이요 웃지 않는 노인은 바보다.', 'speaker': '조지 산타아나'},
 {'content': '사랑은 고생을 면할 수가 없다.그러나 잊을 수 있는 능력이 있다.', 'speaker': '디즈레일리'},
 {'content': '사람을 알려면 그의 지갑,쾌락,그리고 불평을 보라.', 'speaker': '탈무드'},
 {'content': '최고급 회개란 과거의 죄를 청산하고 똑바로 행동하는 것이다.', 'speaker': '윌리암 제임스'},
 {'content': '생활의 기술이란 우리의 환경에 대한 계속적인 적응을 의미한다.', 'speaker': '오카쿠라 카쿠조'},
 {'content': '은혜를 입은 자는 잊지말아야 하고 베푼자는 기억하지 말아야 한다.', 'speaker': '피레 찰론'},
 {'content': '현대화 보다 더 고상한 것이 있다.그것은 바로 영원한 것이다.', 'speaker': '솔로몬 세치터'},
 {'content': '당신은 항상 영웅이 될수 없다.그러나 항상 사람은 될수 있다.', 'speaker': '괴테'},
 {'content': '어떤 가치있는 행동을 하지 아니한 날,그날은 잃은 날이다.', 'speaker': '자콥 보바트'}]


# Connection

In [2]:
# create global connection
from elasticsearch_dsl.connections import connections
connections.create_connection(hosts='localhost', timeout=30)  # global elasticsearch connection

from elasticsearch import Elasticsearch
es = Elasticsearch(hosts='localhost', timeout=30)
print(connections, es)

<elasticsearch_dsl.connections.Connections object at 0x106a7f1d0> <Elasticsearch([{'host': 'localhost'}])>


# Type Mapping (Table Schema)

In [19]:
# noinspection PyUnresolvedReferences
from elasticsearch_dsl import analysis, DocType, String, Date, Integer, analyzer


class FamousSaying(DocType): # Type(Mapping) Class
    # analyzer
    __bage_char_filter = analysis.char_filter('bage_char_filter', 'html_strip', escaped_tags=[])
    __bage_token_filter = analysis.token_filter('bage_token_filter', 'stop', stopwords=[])
    bage_token_analyzer = analyzer('token_analyzer',
                                  type='custom',
                                  char_filter=['html_strip', __bage_char_filter],
                                  tokenizer='whitespace',  # tokens by whitespace
                                  filter=['lowercase', __bage_token_filter],
                              )
    index = 'famous_saying_v2'  # Index(database) for searching
    doc_type = 'famous_saying'  # Type(table)

    # mapping
    speaker = String(index='not_analyzed')
    content = String(analyzer=bage_token_analyzer)
    content_length = Integer(index='not_analyzed')

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.meta.index = self.index
        hash_key = '%s#%s' % (kwargs['speaker'], kwargs['content'])
        self.meta.id = hashlib.md5(hash_key.encode('utf8')).hexdigest()    # primary key
        self.content_length = len(kwargs['content'].split()) # additional field 
        
    @classmethod
    def set_indexing_mode(cls, indexing_mode=True):
        if indexing_mode:
            cls.index = 'famous_saying_v2'  # database for indexing
        else:
            cls.index = 'famous_saying_v2' # database for searching
                
    @classmethod
    def recreate_index(cls, number_of_shards=5, number_of_replicas=1):
        cls.set_indexing_mode()
        ElasticsearchUtil.recreate_index(FamousSaying, number_of_shards, number_of_replicas)

# Indexing (curl)
https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-index_.html

# Indexing (python)
https://elasticsearch-py.readthedocs.io/en/master/

In [6]:
doc = {"speaker" : "박혜웅", "content" : "내가 모르는 것을 아는 척하는 것도 실력이다."}
for i in range(3):
    res = es.index(index=FamousSaying.index, doc_type=FamousSaying.doc_type, body=doc)
    print(res['created'])
    
for i in range(3):
    res = es.index(index=FamousSaying.index, doc_type=FamousSaying.doc_type, id=1, body=doc)
    print(res['created'])

True
True
True
True
False
False


# Manage with ES HQ (Web admin plugin)
http://localhost:9200/_plugin/hq/
- cluster diagram
- cluster health (yellow)
- nodes, shards, documents
- delete index

# Change settings of Index (curl)

# Manage with ES HQ
- cluster health(green)

# Recreate Index

In [11]:
from elasticsearch_dsl import Index
import traceback

class ElasticsearchUtil(object):
    @staticmethod
    def recreate_index(type_class, number_of_shards=5, number_of_replicas=1):
        try:
            print("recreate index... '%s'" % type_class.index)
            new_index = Index(type_class.index)
            new_index.settings(
                number_of_shards=number_of_shards,
                number_of_replicas=number_of_replicas
            )
            new_index.doc_type(type_class)

            try:
                new_index.delete(ignore=404)
            except:
                pass
            new_index.create()
            print('recreate index OK')
        except:
            print(traceback.format_exc())

# Bulk Indexing Tip (python)
https://www.elastic.co/guide/en/elasticsearch/reference/current/indices-update-settings.html

In [12]:
# befor indexing (for performance)
def before_indexing(es):
    FamousSaying.set_indexing_mode(True)
    FamousSaying.recreate_index()
    es.indices.put_settings(body={'refresh_interval': -1, 'number_of_replicas': 0}, index=FamousSaying.index)

In [13]:
# after indexing (for performance)
def after_indexing(es):
    es.indices.put_settings(body={'refresh_interval': '60s', 'number_of_replicas': 1}, index=FamousSaying.index)
    FamousSaying.set_indexing_mode(False)  

# monitoring log messages.
tail -f /usr/local/var/log/elasticsearch.log

# Bulk Indexing (python)

In [15]:
docs

[{'content': '오직 남들을 위하여 산 인생만이 가치 있는 것이다.', 'speaker': '아인슈타인'},
 {'content': '울지 않는 청년은 야만인이요 웃지 않는 노인은 바보다.', 'speaker': '조지 산타아나'},
 {'content': '사랑은 고생을 면할 수가 없다.그러나 잊을 수 있는 능력이 있다.', 'speaker': '디즈레일리'},
 {'content': '사람을 알려면 그의 지갑,쾌락,그리고 불평을 보라.', 'speaker': '탈무드'},
 {'content': '최고급 회개란 과거의 죄를 청산하고 똑바로 행동하는 것이다.', 'speaker': '윌리암 제임스'},
 {'content': '생활의 기술이란 우리의 환경에 대한 계속적인 적응을 의미한다.', 'speaker': '오카쿠라 카쿠조'},
 {'content': '은혜를 입은 자는 잊지말아야 하고 베푼자는 기억하지 말아야 한다.', 'speaker': '피레 찰론'},
 {'content': '현대화 보다 더 고상한 것이 있다.그것은 바로 영원한 것이다.', 'speaker': '솔로몬 세치터'},
 {'content': '당신은 항상 영웅이 될수 없다.그러나 항상 사람은 될수 있다.', 'speaker': '괴테'},
 {'content': '어떤 가치있는 행동을 하지 아니한 날,그날은 잃은 날이다.', 'speaker': '자콥 보바트'},
 {'content': '좋은 일을 많이 해내려고 기다리는 사람은 하나의 좋은 일도 해낼 수가 없다.',
  'speaker': '사무엘 존슨'},
 {'content': '소비된 시간은 존재하고 이용된 시간은 생명이다.', 'speaker': '영'},
 {'content': '내일의 모든 꽃은 오늘의 씨앗에 근거한 것이다.', 'speaker': '중국 속담'},
 {'content': '죄는 취소될 수 없다.용서될 뿐이다.', 'speaker': '스트라빈스키'},
 {'content': '자유느 획득하는 것

In [16]:
from elasticsearch.helpers import bulk
import hashlib

before_indexing(es) # set indexing mode

actions = []
for row in docs: 
    d = FamousSaying(speaker=row['speaker'], content=row['content'])
    actions.append(d.to_dict(include_meta=True))

success, _ = bulk(es, actions, index=FamousSaying.index, raise_on_error=True)

after_indexing(es) # unset indexing mode

print('success docs:', success)

recreate index... 'famous_saying_v2'
recreate index OK
success docs: 1000


# Wait for a moment 

# Searching (Elasticsearch HQ)
- total docs = 1000?
- (공백)
- speaker:"사무엘 존슨"
- content:사람은
- (Sort by Descending)

# Searching (curl)
https://www.elastic.co/guide/en/elasticsearch/reference/current/search-search.html

# Searching (python)

In [21]:
import logging
class Searcher(object):
    def __init__(self, index=FamousSaying.index, doc_type=FamousSaying.doc_type):
        logging.getLogger("urllib3").setLevel(logging.CRITICAL)
        logging.getLogger("requests").setLevel(logging.CRITICAL)
        logging.getLogger('elasticsearch').setLevel(logging.CRITICAL)
        logging.getLogger('elasticsearch.trace').setLevel(logging.CRITICAL)

        self.es = Elasticsearch(hosts='localhost', timeout=300)
        self.index = index
        self.doc_type = doc_type
        
    def escape(self, keyword):     # escaping characters
        keyword = keyword.replace('_', ' ')
        for c in ['.', '%', '+', '-', '=', '&&', '||', '>', '<', '!', '(', ')', '{', '}', '[', ']', '^', '"', '~', '*',
                  '?', ':', '/']:  # '\\',ççç
            keyword = keyword.replace(c, r'\%s' % c)
        return keyword
    
    def search(self, q, field='content'):
        q = '%s:%s' % (field, q) # make query
        r = self.es.search(index=self.index, doc_type=self.doc_type, q=q) # q=Query String Query
        return r
    
    def search_all(self, q, field='content'):
        q = '%s:%s' % (field, q) # make query
        hits = []
        r = self.es.search(index=self.index, doc_type=self.doc_type, q=q) # q=Query String Query
        hits.extend(r['hits']['hits'])
        
        while r['hits']['total'] > len(hits): # need more hits
            _r = self.es.search(index=self.index, doc_type=self.doc_type, q=q, from_=len(hits)) # q=Query String Query
            hits.extend(_r['hits']['hits'])
            r['took'] += _r['took'] # sum elapsed time.
        
        r['hits']['hits']=hits
        return r
    
searcher = Searcher(index=FamousSaying.index, doc_type=FamousSaying.doc_type)

In [23]:
# search not_analyzed field. (term==field)
searcher.search('사무엘 존슨', field='speaker')

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '1a5a8b698307135eb76b3dea101f14d7',
    '_index': 'famous_saying_v2',
    '_score': 0.7063996,
    '_source': {'content': '희망은 어떤 상황에서도 필요하다',
     'content_length': 4,
     'speaker': '사무엘 존슨'},
    '_type': 'famous_saying'},
   {'_id': '31f172fa3e73e006261a952fc65bc8a9',
    '_index': 'famous_saying_v2',
    '_score': 0.57716525,
    '_source': {'content': '모든 소리 중에 음악이 가장 덜 불쾌하다',
     'content_length': 7,
     'speaker': '사무엘 존슨'},
    '_type': 'famous_saying'},
   {'_id': '083b90b2254ae7db8f68c52a5a07c880',
    '_index': 'famous_saying_v2',
    '_score': 0.46265098,
    '_source': {'content': '좋은 일을 많이 해내려고 기다리는 사람은 하나의 좋은 일도 해낼 수가 없다.',
     'content_length': 12,
     'speaker': '사무엘 존슨'},
    '_type': 'famous_saying'}],
  'max_score': 0.7063996,
  'total': 3},
 'timed_out': False,
 'took': 2}

In [24]:
# search not_analyzed field. (term==field)
searcher.search('사무엘', field='speaker')['hits']

{'hits': [], 'max_score': None, 'total': 0}

In [25]:
# search not_analyzed field. (term==field)
searcher.search('존슨', field='speaker')['hits']

{'hits': [], 'max_score': None, 'total': 0}

In [26]:
# search analyzed field. (term!=field)
searcher.search('사람은', field='content')['hits']

{'hits': [{'_id': 'cfa73a1dc3a7cfb090e4a61993e64aee',
   '_index': 'famous_saying_v2',
   '_score': 1.9953598,
   '_source': {'content': '위대한 사람은 모두가 겸손하다.',
    'content_length': 4,
    'speaker': '레싱'},
   '_type': 'famous_saying'},
  {'_id': '449724eee6d39076cf7f956a6f830924',
   '_index': 'famous_saying_v2',
   '_score': 1.825614,
   '_source': {'content': '모든 사람은 죽음 앞에 평등하다',
    'content_length': 5,
    'speaker': '퍼블릴리어스 사이러스'},
   '_type': 'famous_saying'},
  {'_id': 'e0b951aeca496355bfcd8089b9a68b53',
   '_index': 'famous_saying_v2',
   '_score': 1.7325459,
   '_source': {'content': '사람은 스스로 믿는 대로 된다',
    'content_length': 5,
    'speaker': '안톤 체홉'},
   '_type': 'famous_saying'},
  {'_id': 'ff265067d1be4c307963651ce45b2c3c',
   '_index': 'famous_saying_v2',
   '_score': 1.6540035,
   '_source': {'content': '무관심 때문에 사람은 실제로 죽기전에 죽어버린다.',
    'content_length': 6,
    'speaker': '위젤'},
   '_type': 'famous_saying'},
  {'_id': 'a5ec40eaebc51f336cb2584b2c9d83bb',
   '_index': 'famo

In [99]:
searcher.search('사람은', field='content')['hits']['total']

41

In [29]:
print(len(searcher.search('사람은', field='content')['hits']['hits'])) # total != searched docs
print(len(searcher.search_all('사람은', field='content')['hits']['hits'])) # total == searched docs

10
41


In [30]:
# search analyzed field.
searcher.search('좋은 일을 많이 해내려고 기다리는 사람은 하나의 좋은 일도 해낼 수가 없다.', field='content')['hits']

{'hits': [{'_id': '083b90b2254ae7db8f68c52a5a07c880',
   '_index': 'famous_saying_v2',
   '_score': 4.5204167,
   '_source': {'content': '좋은 일을 많이 해내려고 기다리는 사람은 하나의 좋은 일도 해낼 수가 없다.',
    'content_length': 12,
    'speaker': '사무엘 존슨'},
   '_type': 'famous_saying'},
  {'_id': '3eb1146a1bdc115e3de490cd80149c2e',
   '_index': 'famous_saying_v2',
   '_score': 0.36934742,
   '_source': {'content': '핑계를 잘 대는 사람은 거의 좋은 일을 하나도 해내지 못한다.',
    'content_length': 10,
    'speaker': '벤자민 프랭클린'},
   '_type': 'famous_saying'},
  {'_id': '7ad17268626b0e358b80ad49ccdfeed7',
   '_index': 'famous_saying_v2',
   '_score': 0.2435174,
   '_source': {'content': '좋은 나무는 좋은 열매를 맺는다.',
    'content_length': 5,
    'speaker': '랭런드'},
   '_type': 'famous_saying'},
  {'_id': 'fac556cf38ac4875ae3461ee8d67eba4',
   '_index': 'famous_saying_v2',
   '_score': 0.22338933,
   '_source': {'content': '좋은 전쟁 또는 나쁜 평화는 없다.',
    'content_length': 6,
    'speaker': '프랭클린'},
   '_type': 'famous_saying'},
  {'_id': '6356947910e

# Query String Query
https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-query-string-query.html

In [31]:
r = searcher.search_all('"수 있는"', field='content')['hits']
r['total'], r['hits'][-5:]

(25,
 [{'_id': 'd473bf9affcdf123bc3e2ccb70e12ad3',
   '_index': 'famous_saying_v2',
   '_score': 2.0365524,
   '_source': {'content': '인생에 대해 내가 말할 수 있는 전부는, 오 하나님, 즐기라는 거다!',
    'content_length': 11,
    'speaker': '밥 뉴하트'},
   '_type': 'famous_saying'},
  {'_id': 'a9fada90360b1741668e3c5a96cb5df2',
   '_index': 'famous_saying_v2',
   '_score': 1.9323975,
   '_source': {'content': '가치있는 적이 될 수 있는 자는 화해하면 더 가치있는 친구가 될 것이다.',
    'content_length': 12,
    'speaker': '펠담'},
   '_type': 'famous_saying'},
  {'_id': 'de81075004fd35b654f147ab498d14cc',
   '_index': 'famous_saying_v2',
   '_score': 1.8747534,
   '_source': {'content': '중요한 일에 집중할 수 있는 능력이 바로 지능의 가장 결정적인 특징이다',
    'content_length': 11,
    'speaker': '로버트 J. 쉴러'},
   '_type': 'famous_saying'},
  {'_id': 'a1468a5d4b79726e4bff5e168f864fa7',
   '_index': 'famous_saying_v2',
   '_score': 1.6903859,
   '_source': {'content': '휴대할 수 있는 것만 소유하고, 언어, 국가, 사람들을 알아라. 기억을 여행 가방 삼아라',
    'content_length': 13,
    'speaker': '알렉산드르 솔제니친'

In [34]:
r = searcher.search_all('"수" AND "있는"', field='content')['hits']
r['total'], r['hits'][-5:]

(26,
 [{'_id': 'b530b1ea68404898d4b0473ff5ed9212',
   '_index': 'famous_saying_v2',
   '_score': 1.8788158,
   '_source': {'content': '모범은 모든 사람이 읽을 수 있는 교훈이다.',
    'content_length': 7,
    'speaker': '웨스트'},
   '_type': 'famous_saying'},
  {'_id': 'bca4ce31f5b4a0b65ac5017fcba8421f',
   '_index': 'famous_saying_v2',
   '_score': 1.8788158,
   '_source': {'content': '자신을 알 수 있는 사람이야말로 진정한 현인이다.',
    'content_length': 7,
    'speaker': '초서'},
   '_type': 'famous_saying'},
  {'_id': '9d2e331d127036c3e2308c6a4b849dcb',
   '_index': 'famous_saying_v2',
   '_score': 1.7226881,
   '_source': {'content': '교육이 거둘 수 있는 최고의 성과는 관용이다',
    'content_length': 7,
    'speaker': '헬렌 켈러'},
   '_type': 'famous_saying'},
  {'_id': 'fda68beb89f85fd198f8a291d66c1498',
   '_index': 'famous_saying_v2',
   '_score': 1.6945827,
   '_source': {'content': '할 수 있는 자는 행한다. 할 수 없는 자는 가르친다',
    'content_length': 10,
    'speaker': '조지 버나드 쇼'},
   '_type': 'famous_saying'},
  {'_id': '59212831f0e8143f9769384922d48

In [107]:
import json
r1 = set([doc['_source']['content'] for doc in searcher.search_all('"수 있는"', field='content')['hits']['hits']])
r2 = set([doc['_source']['content'] for doc in searcher.search_all('"수" AND "있는"', field='content')['hits']['hits']])
len(r1), len(r2), r2-r1

(25, 26, {'자신이 현명하다고 생각하고 있는 인간은 정녕 구제할 수 없는 바보이다.'})

In [108]:
r = searcher.search_all('"수" OR "없는"', field='content')['hits']
r['total'], r['hits'][-5:]

(97,
 [{'_id': '1fd554bd11db53081e04f124245890bd',
   '_index': 'famous_saying_v1',
   '_score': 0.21365876,
   '_source': {'content': '소유물의 부족은 개선할 수 있으나 영혼의 가난은 해결하기 쉬운 것이 아니다.',
    'content_length': 11,
    'speaker': '몽테뉴'},
   '_type': 'famous_saying'},
  {'_id': 'ab2f1dd1f629321636721df39ee30f42',
   '_index': 'famous_saying_v1',
   '_score': 0.21365876,
   '_source': {'content': '당신이 어떤 위험을 감수하냐를 보면, 당신이 무엇을 가치있게 여기는지 알 수 있다',
    'content_length': 12,
    'speaker': '자넷 윈터슨'},
   '_type': 'famous_saying'},
  {'_id': 'a1468a5d4b79726e4bff5e168f864fa7',
   '_index': 'famous_saying_v1',
   '_score': 0.21365876,
   '_source': {'content': '휴대할 수 있는 것만 소유하고, 언어, 국가, 사람들을 알아라. 기억을 여행 가방 삼아라',
    'content_length': 13,
    'speaker': '알렉산드르 솔제니친'},
   '_type': 'famous_saying'},
  {'_id': 'feb994ed01d1600201ce9218db5ad467',
   '_index': 'famous_saying_v1',
   '_score': 0.21365876,
   '_source': {'content': '지금이 제일 비참하다고 할 수 있는 동안은 아직 제일 비참한 게 아니다',
    'content_length': 12,
    'speake

In [109]:
r = searcher.search_all('"수" "없는"', field='content')['hits']
r['total'], r['hits'][-5:]

(97,
 [{'_id': '1fd554bd11db53081e04f124245890bd',
   '_index': 'famous_saying_v1',
   '_score': 0.21365876,
   '_source': {'content': '소유물의 부족은 개선할 수 있으나 영혼의 가난은 해결하기 쉬운 것이 아니다.',
    'content_length': 11,
    'speaker': '몽테뉴'},
   '_type': 'famous_saying'},
  {'_id': 'ab2f1dd1f629321636721df39ee30f42',
   '_index': 'famous_saying_v1',
   '_score': 0.21365876,
   '_source': {'content': '당신이 어떤 위험을 감수하냐를 보면, 당신이 무엇을 가치있게 여기는지 알 수 있다',
    'content_length': 12,
    'speaker': '자넷 윈터슨'},
   '_type': 'famous_saying'},
  {'_id': 'a1468a5d4b79726e4bff5e168f864fa7',
   '_index': 'famous_saying_v1',
   '_score': 0.21365876,
   '_source': {'content': '휴대할 수 있는 것만 소유하고, 언어, 국가, 사람들을 알아라. 기억을 여행 가방 삼아라',
    'content_length': 13,
    'speaker': '알렉산드르 솔제니친'},
   '_type': 'famous_saying'},
  {'_id': 'feb994ed01d1600201ce9218db5ad467',
   '_index': 'famous_saying_v1',
   '_score': 0.21365876,
   '_source': {'content': '지금이 제일 비참하다고 할 수 있는 동안은 아직 제일 비참한 게 아니다',
    'content_length': 12,
    'speake

In [35]:
# wild card
pprint.pprint(searcher.search_all('지구', field='content')['hits']['hits'])
pprint.pprint(searcher.search_all('지구*', field='content')['hits']['hits'])
pprint.pprint(searcher.search_all('지구?', field='content')['hits']['hits'])

[]
[{'_id': '2b5d8b0773e505fdc32835907015f7d7',
  '_index': 'famous_saying_v2',
  '_score': 1.0,
  '_source': {'content': '지구상에는 인간이외는 위대한 것이 없다.인간에게는 지성이외엔 훌륭한 것이 없다.',
              'content_length': 9,
              'speaker': '헤밀턴'},
  '_type': 'famous_saying'},
 {'_id': 'fc4f6c7cb37af9d7faf4556b351d9d97',
  '_index': 'famous_saying_v2',
  '_score': 1.0,
  '_source': {'content': '천재는 지구가 절대로 길들일 수 없는 전기 같은 힘을 가졌다',
              'content_length': 10,
              'speaker': '리디아 M. 차일드'},
  '_type': 'famous_saying'}]
[{'_id': 'fc4f6c7cb37af9d7faf4556b351d9d97',
  '_index': 'famous_saying_v2',
  '_score': 1.0,
  '_source': {'content': '천재는 지구가 절대로 길들일 수 없는 전기 같은 힘을 가졌다',
              'content_length': 10,
              'speaker': '리디아 M. 차일드'},
  '_type': 'famous_saying'}]


In [111]:
# fuzziness
pprint.pprint(searcher.search_all('지구~1', field='content')['hits']['hits'])

[{'_id': 'b3df16497fd7c2db85c0357c9d72c0d5',
  '_index': 'famous_saying_v1',
  '_score': 2.1111984,
  '_source': {'content': '인생은 다만 한 통의 살구 씨일 뿐이다',
              'content_length': 7,
              'speaker': '로드니 데인저필드'},
  '_type': 'famous_saying'},
 {'_id': '2a01ab42b11bfcca30cc3f84fe0ef817',
  '_index': 'famous_saying_v1',
  '_score': 2.100059,
  '_source': {'content': '지혜 없는 힘은 그 자체의 무게로 쓰러진다',
              'content_length': 7,
              'speaker': '호라티우스'},
  '_type': 'famous_saying'},
 {'_id': '6744fdd39a7ec1bcc412785b14a3d624',
  '_index': 'famous_saying_v1',
  '_score': 1.7372268,
  '_source': {'content': '지혜로운 자는 사랑하고, 다른 모든 이는 욕망할 지니',
              'content_length': 8,
              'speaker': '아프라니우스'},
  '_type': 'famous_saying'},
 {'_id': 'ace44de019469f10782c66c152a5e20d',
  '_index': 'famous_saying_v1',
  '_score': 1.5017462,
  '_source': {'content': '신은 아비가 지은 죄에 대해 자식들을 처벌한다',
              'content_length': 7,
              'speaker': '에우리피데스'},
  '_type': 'fa

In [112]:
# proximity search
pprint.pprint(searcher.search_all('"것이 아니라"~0', field='content')['hits']['hits'])
pprint.pprint(searcher.search_all('"것이 아니라"~1', field='content')['hits']['hits'])

[{'_id': 'f673c6b4af1d0848816f4b2ba7c831f3',
  '_index': 'famous_saying_v1',
  '_score': 3.5504873,
  '_source': {'content': '민주주의는 정지된 것이 아니라 영원히 계속되는 행진이다.',
              'content_length': 7,
              'speaker': '루즈벨트'},
  '_type': 'famous_saying'},
 {'_id': '0c2e591d45e0a7e2bf2a24384bf196e9',
  '_index': 'famous_saying_v1',
  '_score': 2.9587393,
  '_source': {'content': '좋은 집이란 구입하는 것이 아니라 만들어지는 것이어야 한다',
              'content_length': 8,
              'speaker': '조이스 메이나드'},
  '_type': 'famous_saying'},
 {'_id': 'f1fcdc3ccab32ee16b9b2d78485cd50a',
  '_index': 'famous_saying_v1',
  '_score': 2.909992,
  '_source': {'content': '중요한 것은 사랑을 받는 것이 아니라 사랑을 하는 것이었다',
              'content_length': 9,
              'speaker': '서머셋 모옴'},
  '_type': 'famous_saying'}]
[{'_id': 'f673c6b4af1d0848816f4b2ba7c831f3',
  '_index': 'famous_saying_v1',
  '_score': 3.5504873,
  '_source': {'content': '민주주의는 정지된 것이 아니라 영원히 계속되는 행진이다.',
              'content_length': 7,
              'speaker':